# Check Folders
Check whether folders are complete. Also create a list of which ranges are incomplete.

Nicolas Chan, 10/19/2017

In [74]:
# Configuration
results_folder = '/global/scratch/groups/dh/aanderson/results/'
incomplete_list_path = '/global/scratch/groups/dh/aanderson/incomplete.txt'
complete_list_path = '/global/scratch/groups/dh/aanderson/complete.txt'

In [75]:
from os import listdir

incomplete = []
completed = []
consecutive_completed_ranges = [[]]

results_folders = sorted(listdir(results_folder))
for subfolder in results_folders:
    contents = listdir(results_folder + '/' + subfolder)
    # A folder is considered complete if it contains a bighitlist.txt and a smallhitlist.txt
    complete = 'bighitlist.txt' in contents and 'smallhitlist.txt' in contents
    
    if complete:
        completed.append(subfolder)
    else:
        incomplete.append(subfolder)
    
    if complete:
        consecutive_completed_ranges[-1].append(subfolder)
    elif len(consecutive_completed_ranges[-1]) > 0:
        consecutive_completed_ranges.append([])
        
    print(subfolder, 'Complete' if complete else 'INCOMPLETE')

consecutive_completed_ranges = [ r for r in consecutive_completed_ranges if len(r) > 1 ]
print(consecutive_completed_ranges)

Range_141-146 Complete
Results_0-140 INCOMPLETE
Results_147-151 INCOMPLETE
Results_152-153 INCOMPLETE
[]


In [76]:
# Write list of incomplete ranges
import datetime

incomplete_list = open(incomplete_list_path, 'w')
incomplete_list.write('# Generated ' + str(datetime.datetime.now()) + '\n')
for inc in incomplete:
    incomplete_list.write(inc + '\n')
incomplete_list.close()

In [72]:
# Write list of complete ranges
complete_list = open(complete_list_path, 'w')
complete_list.write('# Generated ' + str(datetime.datetime.now()) + '\n')
for c in completed:
    complete_list.write(c + '\n')
complete_list.close()

In [77]:
# concatenate_files based on https://stackoverflow.com/a/13613375/8706910
def concatenate_files(files, output_file):
    with open(output_file, 'w', encoding='utf-8') as output:
        for file in files:
            with open(file, encoding='utf-8') as input_file:
                for line in input_file:
                    output.write(line)
                    
def results_folder_to_range(folder_name):
    start, end = folder_name.split('_')[1].split('-')
    return int(start), int(end)

import os
import sys
import errno

# dir-create copied from AdamAndersonFindSumerianWorkflow
def dir_create(path):
    try:
        os.makedirs(path)
    except OSError as exception:
        if exception.errno != errno.EEXIST:
            raise
        else:
            print('Folder at: ' + path + ' already exists. Skipping...')

import shutil
# Merge all consecutive completed ranges into a single range
for completed_range in consecutive_completed_ranges:
    start_index = None
    end_index = None
    for folder in completed_range:
        start, end = results_folder_to_range(folder)
        start_index = start if start_index == None else min(start_index, start)
        end_index = end if end_index == None else max(end_index, end)
    
    smallhits_paths = [ results_folder + folder + '/smallhitlist.txt' for folder in completed_range ]
    bighits_paths = [ results_folder + folder + '/bighitlist.txt' for folder in completed_range ]
    
    merge_folder = results_folder + 'Results_' + str(start_index) + '-' + str(end_index)
    dir_create(merge_folder)
    concatenate_files(smallhits_paths, merge_folder + '/smallhitlist.txt')
    concatenate_files(bighits_paths, merge_folder + '/bighitlist.txt')
    print('Merged Range', start_index, end_index)
    
    # Delete old folders
    # WARNING: This hasn't been fully tested. Please test that the merge works correctly before deleting folders.
    for folder in completed_range:
        shutil.rmtree(results_folder + folder) # Dangerous!
    
    